In [1]:
import torch
from torchvision import transforms
import glob
import cv2
import matplotlib.pyplot as plt
import pandas as pd
import torchvision

In [10]:
from PIL import Image
import requests
from io import BytesIO

In [3]:
#Importing the data
data=pd.read_csv("https://raw.githubusercontent.com/PranavM98/Facial-Detection-and-Emotion-Classifier/main/00_Data/01_Transformed_Data/face_detection_data.csv")

In [4]:
data.head()

,Filename,xmin,ymin,xmax,ymax,label
0,Kevin_Gil_0001.jpg,71,69,180,178,1
1,Gerry_Kelly_0001.jpg,72,71,176,175,1
2,Eliane_Karp_0003.jpg,60,66,183,189,1
3,Vojislav_Kostunica_0006.jpg,66,67,183,184,1
4,Phil_Johnson_0001.jpg,64,64,186,186,1


In [11]:
#Storing the tensor images
images=[]

#Storing the targets - boxes and labels
targets=[]

#Convert image to tensor
convert_tensor = transforms.ToTensor()

#Iterating over all images in dataset
for i, r in data.iterrows():
    d={}
    x1= data.loc[i,'xmin']
    y1= data.loc[i,'ymin']
    x2= data.loc[i,'xmax']
    y2= data.loc[i,'ymax']

    boxes=[x1,y1,x2,y2]    
    labels=1

    d['boxes']=torch.FloatTensor(boxes).reshape(1,4)
    d['labels']=labels
    
    targets.append(d)

    # Recreate file-path
    image_file_path = data.loc[i,'Filename']
    image_file_path = 'https://raw.githubusercontent.com/PranavM98/Facial-Detection-and-Emotion-Classifier/main/00_Data/01_Transformed_Data/Face_Detection_Images/' + image_file_path

    response = requests.get(image_file_path)
    img = Image.open(BytesIO(response.content))

    #img = cv2.imread(image_file_path)
    a=convert_tensor(img)
    images.append(a)


KeyboardInterrupt: ignored

In [ ]:
train_val_split_val = int(len(images) * 0.80)

train_images = images[:train_val_split_val]
train_targets = targets[:train_val_split_val]

test_images = images[train_val_split_val:]
test_targets = targets[train_val_split_val:]

In [ ]:
print('The length of entire dataset is: ', len(images))
print('80-20 Split Train dataset size of: ', train_val_split_val)
print('The length of Train dataset is: ', len(train_images))
print('The length of Test dataset is: ', len(test_images))

In [ ]:
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

In [ ]:
output = model(test_images, test_targets)

In [ ]:
model.eval()

In [ ]:
torch.save(model.state_dict('../03_Model_Checkpoints/object_detection_checkpoint1.pth'))